In [18]:
from package.sphincs import Sphincs
from tqdm import tqdm

### Instantiate a $SPHINCS^+$ Object and Setting parameters ###

We will be using :
   - Security Parameter: $n=2$
   - Winternitz Parameter: $w=16$
   - Hypertree Height: $h=4$
   - Hypertree Layers: $d=2$
   - $FORS$ Trees Number: $k=4$
   - $FORS$ Trees Height: $a=2$

In [19]:
sphincs = Sphincs()

# sphincs.set_n(2)
# sphincs.set_h(4)
# sphincs.set_d(2)
# sphincs.set_k(4)
# sphincs.set_a(2)

sphincs.set_w(4)
sphincs.set_hypertree_height(32)

### Generating a Key Pair ###

In [20]:
sk, pk = sphincs.generate_key_pair()
print("Secret Key: ", sk)
print()
print("Public Key: ", pk)

Secret Key:  b'\x85!}4K&\xe9\x14\xbc\x89k\xd1\xde;\x01<\x05\xc4\x9eq\xd0\xc6\xfb\xef1\xd0O?\rJ\x89\xeeW\xdc\x8c\xd0\xf4\xd8z\x06\x9b\xd0i<\xa2\x13\x1f\xcfYc\xaeg\xda\x911g\xbb\xeb\x99\x06\xd4\xe0\xe9\xd36\x89`\x01;\x9c%\x9dw\x02A+\xe2x\xd5v\xe6{En\xf9\x98\xdbl\x05\xdb\x9e\x05\xafg\x8aK\xa6x\x01\xf9=Y\x16\x01\x91\xbd|\x93\xe0\xbf\x18\x92P\x1a\xe7Z\x9a$\xcfk{\x82\xcc\xdb\xdf\x9c\x16\xb9'

Public Key:  b'6\x89`\x01;\x9c%\x9dw\x02A+\xe2x\xd5v\xe6{En\xf9\x98\xdbl\x05\xdb\x9e\x05\xafg\x8aK\xa6x\x01\xf9=Y\x16\x01\x91\xbd|\x93\xe0\xbf\x18\x92P\x1a\xe7Z\x9a$\xcfk{\x82\xcc\xdb\xdf\x9c\x16\xb9'


### Signing M ###

In [21]:
m = b'Ripples of paradox spread out across the sea of causality.'

signature = sphincs.sign(m, sk)

print("Signature Size: ", len(signature))

Signature Size:  7684


In [16]:
print("Is signature Correct ? ", sphincs.verify(signature, m, pk))

Is signature Correct ?  False


In [17]:
sphincs.verify(signature, m, pk)


False

### Trying to find secret key with a Brute Force Attack on Secret Key###

In [7]:
sk_crack = bytes()

for i in tqdm(range(0, 2 ** (sphincs._n * 8))):
    sk_crack = i.to_bytes(sphincs._n, 'big')  # Secret Key
    
    sk_crack += bytes(sphincs._n)   # Random Secret PRF, important to prevent forged messages from actual messages
                                    # But Because we are brute forcing Secret Key, messages are forged before
                                    # We don't need to create a really random one (0 is fine)
    sk_crack += pk  # Public Key
    
    sig_crack = sphincs.sign(m, sk_crack)  # Creating a signature
    
    if sphincs.verify(sig_crack, m, pk):  # Check if signature could be trust with the Public Key
        print("Secret Key Found: ", sk_crack, "\nWith main Private Seed: ", sk_crack[:sphincs._n])
        print("Cycles: ", i)
        break

print("\nDid we found the actual Private Seed? ", sk[:sphincs._n] == sk_crack[:sphincs._n])

if sk[:sphincs._n] != sk_crack[:sphincs._n]:
    print("We found a collision with the main seed!")

  0%|                                                                             | 47/65536 [00:13<5:06:02,  3.57it/s]


KeyboardInterrupt: 

### Trying to forge message with found Key ###

In [ ]:
m2 = b'The pen is mightier than the sword ... if the sword is very short, and the pen is very sharp.'

signature2 = sphincs.sign(m2, sk_crack)

print(len(signature2))

58


In [ ]:
print("Is signature Correct ? ", sphincs.verify(signature2, m, pk))

Is signature Correct ?  False


Our signature is wrong because we tried finding the secret key using only $m$ signature, with $m_2$, this kind of collision doesn't work !
We need to find the real secret key in order to forge our own message, and so test every possibilities.